In [ ]:
# Field Completeness by Object Type (Top 25)

In [1]:
import glob
import pandas as pd

df = pd.concat(map(pd.read_csv, glob.glob('/home/richard/Git/metadata/vam-metadata/data/*.csv')))

In [2]:
# First create new data frame reducing down to just the columns we need

complete_df = df.filter(regex='\[objectType|completeness*', axis=1)
complete_df

,[objectType,completeness:TOTAL,completeness:COMPOSITION,completeness:PRODUCTION,completeness:ASSOCIATION,completeness:MATERIALITY,completeness:MUSEUM,completeness:HISTORY,completeness:PHYSICALITY,completeness:ACADEMIC,completeness:DEPICTION
0,Print,0.025641,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Print,0.025641,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Print,0.025641,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Print,0.025641,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Print,0.025641,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
10913,Fashion design,0.282051,1.0,0.571429,0.0,0.5,1.0,0.0,0.5,0.0,0.0
10914,Fashion design,0.282051,1.0,0.571429,0.0,0.5,1.0,0.0,0.5,0.0,0.0
10915,Fashion design,0.282051,1.0,0.571429,0.0,0.5,1.0,0.0,0.5,0.0,0.0
10916,Fashion design,0.282051,1.0,0.571429,0.0,0.5,1.0,0.0,0.5,0.0,0.0


In [5]:
top_25_objects = complete_df['[objectType'].value_counts()[:25].index.tolist()

In [3]:
import re
tidied_df = complete_df.rename(columns=lambda x: re.sub('completeness:','',x))
tidied_df

,[objectType,TOTAL,COMPOSITION,PRODUCTION,ASSOCIATION,MATERIALITY,MUSEUM,HISTORY,PHYSICALITY,ACADEMIC,DEPICTION
0,Print,0.025641,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Print,0.025641,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Print,0.025641,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Print,0.025641,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Print,0.025641,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
10913,Fashion design,0.282051,1.0,0.571429,0.0,0.5,1.0,0.0,0.5,0.0,0.0
10914,Fashion design,0.282051,1.0,0.571429,0.0,0.5,1.0,0.0,0.5,0.0,0.0
10915,Fashion design,0.282051,1.0,0.571429,0.0,0.5,1.0,0.0,0.5,0.0,0.0
10916,Fashion design,0.282051,1.0,0.571429,0.0,0.5,1.0,0.0,0.5,0.0,0.0


In [6]:
# Save all the possible collectioncode
import altair as alt

ranges = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
count =0

dept_col_counts = None
charts = []
hcharts = []
        
for obj_type in top_25_objects:
  dept_rows_df = pd.DataFrame()
  dept_stats_df = tidied_df[tidied_df['[objectType'] == obj_type] 
  # Now need to loop over each completeness coll in turn
  for column in dept_stats_df.columns[1:]:
    # Group into counts of percentages 0-10, 10-20, ... 90-100
    # dept_col_counts = dept_stats_df[column].value_counts(bins=10, sort=False)
    
    dept_col_counts = dept_stats_df[column].groupby(pd.cut(dept_stats_df[column], ranges, labels=["0%","10%","20%","30%","40%","50%","60%","70%","80%","90%"], include_lowest=True)).count()
    dept_col_counts.name = column
    dept_rows_df = dept_rows_df.append(dept_col_counts, ignore_index=False)
    
  dept_rows_df = dept_rows_df.rename_axis('Concept').rename_axis('Percentages', axis='columns')
    # TO handle converting from CategoricalIndex - may change https://github.com/pandas-dev/pandas/issues/19136
  dept_rows_df.columns = dept_rows_df.columns.tolist()
  #print(dept_rows_df)
  dept_rows_df = dept_rows_df.reset_index()

  dept_rows_melt_df = dept_rows_df.melt(id_vars=['Concept'])

  chart = alt.Chart(dept_rows_melt_df).mark_bar().encode(
       x='value:Q',
       y='Concept:O',
        color='variable:O'
    ).properties(width=220, title="%s (%d)" % (obj_type, len(dept_stats_df)))
  charts.append(chart)
  if count > 3:
      hcharts.append(alt.hconcat(*charts))
      charts = []
      count = 0
  else:
      count += 1

hcharts.append(alt.hconcat(*charts))

big_chart = alt.vconcat(*hcharts)

In [8]:
big_chart.save('type-field.completeness.png')

ValueError: No enabled saver found that supports format='png'